In [77]:
import os
from PIL import Image
from tqdm import tqdm
from transformers import (
    BlipProcessor,
    BlipForConditionalGeneration,
)
import torch
import pandas as pd

In [78]:
from config_file import config

In [79]:
# Путь к папке с изображениями
images_folder = config.IMAGES_PATH
output_file = config.DATA_PATH / "annotations.txt"

# ⚙️ Устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
output_file = "annotations.txt"

In [80]:
# 🔄 Загрузка модели и процессора один раз
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base"
).to(device)

In [81]:
def generate_caption(image_path: str, text: str, game: pd.Series) -> str:
    try:
        image = Image.open(image_path).convert("RGB")
        inputs = processor(images=image, text=text, return_tensors="pt").to(device)
        output = model.generate(**inputs)
        caption = processor.decode(output[0], skip_special_tokens=True)

        genres = "Unknown genres" if pd.isna(game["genres"]) else game["genres"]
        perspectives = (
            "Unknown perspectives"
            if pd.isna(game["player_perspectives"])
            else game["player_perspectives"]
        )
        caption = f"Game has genres: {genres}; perspectives: {perspectives}. " + caption

        return caption
    except Exception as e:
        print(f"Ошибка при обработке {image_path}: {e}")
        return ""

In [82]:
df = pd.read_csv(config.DATA_PATH / "similar_games.csv")

In [ ]:
# 📷 Обработка изображений
image_files = os.listdir(images_folder)
annotations = {}
text = "This game screenshot shows"

for i in tqdm(range(len(df)), desc="Генерация аннотаций"):
    game = df.iloc[i]
    img_file = game["image_id"] + ".jpg"
    image_path = images_folder / img_file
    caption = generate_caption(
        image_path,
        text,
        game,
    )

    annotations[game["image_id"]] = caption

Генерация аннотаций: 100%|██████████| 2561/2561 [14:01<00:00,  3.04it/s]

✅ Аннотации сохранены в annotations.txt


In [89]:
with open("annotations.txt", "r") as f:
    lines = f.readlines()
    lines = map(lambda line: line.strip().split("\t"), lines)

In [ ]:
import pandas as pd

annotations = pd.read_csv("annotations.txt", sep="\t", header=None)
annotations.columns = ["image_id", "annotation"]
annotations.head()

,image_id,annotation
0,koajaesby7cmhujlcwkl.jpg,"Game has genres: Shooter, Platform, Puzzle, Ad..."
1,poegvjadtys8fflxpxnu.jpg,"Game has genres: Shooter, Platform, Puzzle, Ad..."
2,lrx26njeiciksjqby7ou.jpg,"Game has genres: Shooter, Platform, Puzzle, Ad..."
3,vux5gzepdeqvhvm84rcz.jpg,"Game has genres: Shooter, Platform, Puzzle, Ad..."
4,jlt7ncnm7cheka0hmypd.jpg,"Game has genres: Shooter, Platform, Puzzle, Ad..."


In [97]:
annotations.to_csv(config.DATA_PATH / "annotations.csv", index=False)